In [ ]:
!gdown --id '10Pq5dtzYOujE5wzWAG-JGw_ocAoQXsN_'
!unzip -qq /content/ai-hackathon-online-2021-qa-scg.zip -d 'QA-SCG'

Downloading...
From: https://drive.google.com/uc?id=10Pq5dtzYOujE5wzWAG-JGw_ocAoQXsN_
To: /content/ai-hackathon-online-2021-qa-scg.zip
100% 488k/488k [00:00<00:00, 65.8MB/s]
replace QA-SCG/SCG-Train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
import os
import pandas as pd

# SCG dataset

In [ ]:
train = pd.read_csv('/content/QA-SCG/SCG-Train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_path  297 non-null    object
 1   question      297 non-null    object
 2   answer        297 non-null    object
dtypes: object(3)
memory usage: 7.1+ KB


In [ ]:
train.columns = ['article_id', 'question', 'answer']

In [ ]:
train

,article_id,question,answer
0,SCG102,ตัวแทนจาก WEDO Developer “Team 26” ชื่ออะไร?,“Team 26”
1,SCG102,มีกี่ประเทศในเอเชียตะวันออกเฉียงใต้ที่แข่งขันใ...,10
2,SCG102,การแข่งขันมีชื่อว่าอะไร,AWS Microservices GameDay 2020
3,SCG102,แข่งขันเกี่ยวกับอะไร,แข่งกันตั้งระบบจากโจทย์ที่ได้รับมาให้เสถียรและ...
4,SCG102,ใครได้รับชัยชนะในการแข่งขันนี้,ตัวแทนจาก WEDO Developer “Team 26”
...,...,...,...
292,SCG216,คุณค่าที่ผู้เขียนบทความได้จากการทำงานมากที่สุด...,การทำภาพใหญ่ หรือ corporate startegy และโอกาสก...
293,SCG216,ข้อดีของการทำ strategic partnerships คืออะไร,เราจะคิดเองเออเองคนเดียวไม่ได้ ทุกส่วนอาศัยประ...
294,SCG216,Value Proposition ในวิธีคิดฉบับ Digital Office...,เพื่อให้แน่ใจว่าสิ่งที่กำลังจะทำตอบโจทย์ users...
295,SCG216,ตัวอย่างระบบการดูแลพนักงานที่ Digital Office ม...,Health center ใจกลางบริษัทที่จะเดินไปหาหมอเมื่...


In [ ]:
temp = []
file_path = '/content/QA-SCG/train'
for each_file in os.listdir(file_path):
    with open(os.path.join(file_path, each_file),
              mode='r', encoding='utf-8') as f:
        temp.append({'context' : f.read(), 'article_id' : each_file.replace('.txt', '')})
SCG_documents = pd.DataFrame(temp)
SCG_documents.head()

,context,article_id
0,📌 ถอดรหัส 'วิถีปกติใหม่' อยู่อย่างไรให้รอดและร...,SCG121
1,ตั้งการ์ดรับมือ COVID-19 ให้สุขภาพแล้ว อย่าลืม...,SCG104
2,﻿Creative Talk\n15 มกราคม 2020\n \n \nCreative...,SCG205
3,📌 หากคุณอยากเปลี่ยนแปลงตัวเองให้เป็น talent ที...,SCG126
4,เรียนยังไงให้คว้า “งานแห่งอนาคต” ได้?\n.\nในยุ...,SCG137


In [ ]:
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
SCG_documents['context'] = SCG_documents['context'].map(lambda x: deEmojify(x))
dataset = train.join(SCG_documents.set_index('article_id'), on='article_id')
dataset.head(3)

,article_id,question,answer,context
0,SCG102,ตัวแทนจาก WEDO Developer “Team 26” ชื่ออะไร?,“Team 26”,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...
1,SCG102,มีกี่ประเทศในเอเชียตะวันออกเฉียงใต้ที่แข่งขันใ...,10,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...
2,SCG102,การแข่งขันมีชื่อว่าอะไร,AWS Microservices GameDay 2020,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...


## First Try

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece #some pre-trained models require this tokenized

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [ ]:
# Load pretrained from Hugging Face
pretrained = "deepset/xlm-roberta-large-squad2"
tokenizer = AutoTokenizer.from_pretrained(pretrained)
model = AutoModelForQuestionAnswering.from_pretrained(pretrained)

In [ ]:
from transformers import pipeline
from tqdm.notebook import tqdm

In [ ]:
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [ ]:
# for faster
!gdown --id '1vtS1Ij963CPKjTPg_KmY0874aNQS6Fqq'
dataset = pd.read_csv('/content/scg_first_try.csv')
# for tup in tqdm(dataset.itertuples(), total=dataset.shape[0]):
#     q = tup.question
#     c = tup.context
#     dataset.loc[tup.Index, 'predict'] = pipe(question=q, context=c)['answer']

Downloading...
From: https://drive.google.com/uc?id=1vtS1Ij963CPKjTPg_KmY0874aNQS6Fqq
To: /content/scg_first_try.csv
100% 2.10M/2.10M [00:00<00:00, 133MB/s]


In [ ]:
dataset

,article_id,question,answer,context,predict
0,SCG102,ตัวแทนจาก WEDO Developer “Team 26” ชื่ออะไร?,“Team 26”,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,\n#WEDO #Developer
1,SCG102,มีกี่ประเทศในเอเชียตะวันออกเฉียงใต้ที่แข่งขันใ...,10,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,10
2,SCG102,การแข่งขันมีชื่อว่าอะไร,AWS Microservices GameDay 2020,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,\nAWS Microservices GameDay 2020
3,SCG102,แข่งขันเกี่ยวกับอะไร,แข่งกันตั้งระบบจากโจทย์ที่ได้รับมาให้เสถียรและ...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,โดยแข่งกันตั้งระบบจากโจทย์ที่ได้รับมาให้เสถีย...
4,SCG102,ใครได้รับชัยชนะในการแข่งขันนี้,ตัวแทนจาก WEDO Developer “Team 26”,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,ตัวแทนจาก WEDO Developer “Team 26”
...,...,...,...,...,...
292,SCG216,คุณค่าที่ผู้เขียนบทความได้จากการทำงานมากที่สุด...,การทำภาพใหญ่ หรือ corporate startegy และโอกาสก...,﻿1 ปีที่ Digital Office ฉันมาทำอะไรที่นี่\n\nP...,และโอกาสการ consult จากพี่ๆคือสิ่งที่เป็น
293,SCG216,ข้อดีของการทำ strategic partnerships คืออะไร,เราจะคิดเองเออเองคนเดียวไม่ได้ ทุกส่วนอาศัยประ...,﻿1 ปีที่ Digital Office ฉันมาทำอะไรที่นี่\n\nP...,คือเราจะคิดเองเออเองคนเดียวไม่ได้
294,SCG216,Value Proposition ในวิธีคิดฉบับ Digital Office...,เพื่อให้แน่ใจว่าสิ่งที่กำลังจะทำตอบโจทย์ users...,﻿1 ปีที่ Digital Office ฉันมาทำอะไรที่นี่\n\nP...,เพื่อให้แน่ใจว่าสิ่งที่กำลังจะทำตอบโจทย์ user...
295,SCG216,ตัวอย่างระบบการดูแลพนักงานที่ Digital Office ม...,Health center ใจกลางบริษัทที่จะเดินไปหาหมอเมื่...,﻿1 ปีที่ Digital Office ฉันมาทำอะไรที่นี่\n\nP...,Health center ใจกลางบริษัทที่จะเดินไปหาหมอเมื...


## Extract Hashtags

In [ ]:
def extract_hashtags(text):
    result = []
    for token in text.split():
        if token.startswith('#'):
            result.append(token)
    return (' '.join(result)).strip()

In [ ]:
dataset['hashtag'] = dataset.context.map(lambda x: extract_hashtags(x))

In [ ]:
dataset.head()

,article_id,question,answer,context,predict,hashtag
0,SCG102,ตัวแทนจาก WEDO Developer “Team 26” ชื่ออะไร?,“Team 26”,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,\n#WEDO #Developer,#WEDO #Developer #AWS #AWSGameDay2020
1,SCG102,มีกี่ประเทศในเอเชียตะวันออกเฉียงใต้ที่แข่งขันใ...,10,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,10,#WEDO #Developer #AWS #AWSGameDay2020
2,SCG102,การแข่งขันมีชื่อว่าอะไร,AWS Microservices GameDay 2020,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,\nAWS Microservices GameDay 2020,#WEDO #Developer #AWS #AWSGameDay2020
3,SCG102,แข่งขันเกี่ยวกับอะไร,แข่งกันตั้งระบบจากโจทย์ที่ได้รับมาให้เสถียรและ...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,โดยแข่งกันตั้งระบบจากโจทย์ที่ได้รับมาให้เสถีย...,#WEDO #Developer #AWS #AWSGameDay2020
4,SCG102,ใครได้รับชัยชนะในการแข่งขันนี้,ตัวแทนจาก WEDO Developer “Team 26”,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,ตัวแทนจาก WEDO Developer “Team 26”,#WEDO #Developer #AWS #AWSGameDay2020


In [ ]:
!pip install pythainlp -qq
!pip install deepcut -qq

     |████████████████████████████████| 2.0 MB 9.7 MB/s 


In [ ]:
from pythainlp import word_tokenize

In [ ]:
def word_count(text, counts={}):
    tokens = word_tokenize(text, engine='deepcut')
    for token in tokens:
        if token not in counts.keys():
            counts[token] = 1
        else:
            counts[token] += 1
    return counts

counts = {}
for tuple_ in dataset.itertuples(index=True):
    if tuple_.answer.strip().startswith('#'):
        word_count(tuple_.question, counts)

counts

{' ': 43,
 ' #DigitalOffice': 1,
 '#GoBigRed': 2,
 '#WEDO': 5,
 '?': 22,
 'hashtag': 14,
 'ความ': 2,
 'จาก': 9,
 'ด้วย': 7,
 'นอก': 6,
 'นี้': 4,
 'บท': 2,
 'บทความ': 21,
 'บ้าง': 23,
 'ประกอบ': 7,
 'ผู้': 4,
 'มี': 12,
 'ยัง': 1,
 'ลง': 4,
 'อะไร': 23,
 'อีก': 1,
 'เขียน': 4,
 'แฮชแท็ก': 10,
 'ใน': 15,
 'ใส่': 4,
 'ไป': 4}

In [ ]:
hashtags = ['hashtag', 'แฮชแท็ก']

In [ ]:
for tuple_ in dataset.itertuples():
    for h in hashtags: 
        if h in tuple_.question:
            dataset.loc[tuple_.Index, 'predict_new'] = tuple_.hashtag

In [ ]:
dataset.loc[~dataset.predict_new.isna()]

,article_id,question,answer,context,predict,hashtag,predict_new
7,SCG102,มีแฮชแท็กอะไรบ้างจากบทความ,#WEDO #Developer #AWS #AWSGameDay2020,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,มีทิปส์อะไรที่ชาว Dev ต้องรู้!,#WEDO #Developer #AWS #AWSGameDay2020,#WEDO #Developer #AWS #AWSGameDay2020
11,SCG103,จากบทความ ประกอบด้วย hashtag อะไรบ้าง?,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent,อยากเป็น WEDO Young Talent เพื่อไปลุยโปรแกรมทำ...,#GoBigRed,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent
20,SCG104,มี hashtag อะไรบ้างในบทความ?,#WEDO #NewNormal,ตั้งการ์ดรับมือ COVID-19 ให้สุขภาพแล้ว อย่าลืม...,\n#WEDO #NewNormal,#WEDO #NewNormal,#WEDO #NewNormal
26,SCG106,ข้อความอะไรคือส่วนขยายของ marketing hashtag #W...,"we hire for attitude, we train for skill.",“ชุดความคิดและทัศนคติ” สำคัญยิ่งกว่าความเก่ง\n...,"we hire for attitude, we train for skill.",#WEDO #talent #Intrepreneur,#WEDO #talent #Intrepreneur
30,SCG107,มีแฮชแท็กอะไรบ้างจากบทความ?,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent,ขอบคุณทุกความสนใจจากเหล่าคนรุ่นใหม่ ที่อยากมา...,#GoBigRed,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent
35,SCG108,บทความผู้เขียนใส่แฮชแท็กอะไรลงไปบ้าง?,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent,Final call WEDO Young Talent Program 2021\n\n...,#GoBigRed,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent
40,SCG110,นอกจาก hashtag #WEDO ยังมี hashtag อะไรอีกบ้าง...,#DigitalOffice #GoBigRed #FutureTalent #Softwa...,ด่วน! WEDO ทีมงาน Corporate Innovation ไฟแรงแ...,\n#DigitalOffice,#WEDO #DigitalOffice #GoBigRed #FutureTalent #...,#WEDO #DigitalOffice #GoBigRed #FutureTalent #...
47,SCG113,hashtag ในบทความประกอบด้วยอะไรบ้าง?,#WEDO #DigitalOffice #GoBigRed #WEDOxHarbourSp...,นี่คือการเฟ้นหา Data Science talent ที่ฉีกทุกต...,แต่ต้องเข้าใจผู้คนเป็นอย่างดี,#WEDO #DigitalOffice #GoBigRed #WEDOxHarbourSp...,#WEDO #DigitalOffice #GoBigRed #WEDOxHarbourSp...
61,SCG114,hashtag ในบทความประกอบด้วยอะไรบ้าง?,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent,โอกาสมาถึงแล้ว! ถึงเวลาที่นิสิต นักศึกษาทุกชั...,Digital Content & Marketing,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent,#WEDO #DigitalOffice #GoBigRed #WEDOYoungTalent
65,SCG116,บทความผู้เขียนใส่แฮชแท็กอะไรลงไปบ้าง?,#WEDO #DigitalOffice #GoBigRed #WEDOxHarbourSp...,"“ถึงเวลาพิชิตบททดสอบ!! สู่ปลายทาง ""Data Scienc...",#GoBigRed #WEDOxHarbourSpace #AuditionDay2020,#WEDO #DigitalOffice #GoBigRed #WEDOxHarbourSp...,#WEDO #DigitalOffice #GoBigRed #WEDOxHarbourSp...
